### Callin Switzer

2 Nov 2016

-Read in all the Kalmia polygon images (from R KalmiaVisualizePolygons_V2.R)
- Make sure to use Python [conda env:vis_env]

-visualize in 3D

-Resample all of the images

-Stack resampled images (in 3D)

-Visualize a heatmap

-Save heatmap

In [1]:
import cv2

In [2]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

In [3]:
# scikit image
import skimage
from skimage import io

In [4]:
import os
import pandas as pd
import time

In [5]:
import scipy
import scipy.ndimage

from mpl_toolkits.mplot3d import Axes3D
import sys

In [ ]:
from mayavi import mlab
from scipy import stats

In [ ]:
% qtconsole

May want to look at this: http://stackoverflow.com/questions/25286811/how-to-plot-a-3d-density-map-in-python-with-matplotlib

In [ ]:
# list files in directory

In [6]:
os.chdir("/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/KalmiaExamples/KalmiaManualTrig_VidsToProcess")

In [7]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
photoList = [ x for x in onlyfiles if not x.startswith('.') and x.endswith('.png')]

In [8]:
len(photoList)

29

In [414]:
# read in image and resize
kk = 1
# img = np.array(cv2.imread(photoList[kk], 0))
img = np.array(cv2.imread('20160615_153850.png', 0))

newSideLen = 200

r = float(newSideLen) / img.shape[1]
dim = (int(newSideLen), int(img.shape[0] * r))
 
# perform the actual resizing of the image and show it
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

resized = np.array((resized < 255) * 1.0)

# clear edges of resized image (grid from graph)
resized[:, 0:newSideLen/20] = 0
resized[newSideLen - newSideLen/20:newSideLen, :] = 0

plt.close('all')
io.imshow(resized)
thismanager = plt.get_current_fig_manager()
thismanager.window.wm_geometry("-1-1")




''

In [415]:
plt.imshow(resized)

In [435]:
# make empty np array
# for future reference, these are stacked wrong

# rotate image for visualization with numpy
resized_flip = np.rot90(resized, 3)
stackedArr = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0

midpt = stackedArr.shape[1]/2

stackedArr[midpt,:,:] = stackedArr[midpt,:,:] + resized_flip

stackedArr[:, midpt, :] = stackedArr[:, midpt, :] + resized_flip

# get size
sys.getsizeof(stackedArr)/1000/1000 # in MB


0

In [ ]:
x,y,z = stackedArr.nonzero()

In [ ]:
#plt.close('all')
plt.clf()
fig = plt.figure(1)
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, zdir='z', c = stackedArr[x,y,z], 
           cmap = 'Greys', edgecolor = 'none', marker = 's')
#plt.savefig("demo.png")

In [ ]:
plt.close('all')
mlab.close(all = True)

In [ ]:
# visualize with MayaVi points in 3d
x,y,z = stackedArr.nonzero()
xyz = np.vstack([x, y, z,])
kde = stats.gaussian_kde(xyz)
density = kde(xyz)

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')
pts = mlab.points3d(x, y, z, stackedArr[x,y,z], colormap = 'hot', 
                    scale_mode='none', scale_factor=1, opacity = 0.2)
mlab.axes()
mlab.show()

In [433]:
# visualize density with MayaVi
density = stackedArr
grid = mlab.pipeline.scalar_field(density)
min1 = density.min()
max1=density.max()
mlab.pipeline.volume(grid, vmin=min1, vmax=min1 + .5*(max1-min1))

mlab.axes()
mlab.show()

# Rotate image

In [436]:
plt.clf()
io.imshow(resized)

In [437]:
# rotate image with opencv
w = resized.shape[0]
h = resized.shape[1]
M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36, 1.0)
rotated = cv2.warpAffine(resized, M, (w, h))
io.imshow(rotated < .4)


In [438]:
# rotate with scipy
plt.clf()
i2 = scipy.ndimage.interpolation.rotate(np.array(resized), 36, axes=(1,0))
io.imshow(i2)

In [439]:
stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
stackedRot[midpt,:,:] = stackedRot[midpt,:,:] + resized_flip

In [444]:
# visualize
mlab.close(all = True)
mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(stackedRot)
mlab.axes()

In [429]:
mlab.clf()
#pixels (should be an even number to make it symmetrical)

pollWidth = newSideLen / 100 / 2

stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] = stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] + resized_flip



# pretty fast
w,h = stackedRot.shape[:2]
M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36, 1.0)
rotated = cv2.warpAffine(resized, M, (w, h))


for ii in range(newSideLen):
    stackedRot[:,:,ii] = ((cv2.warpAffine(stackedRot[:,:,ii], M, (w, h)) +
                          stackedRot[:,:,ii]) < 1 ) * 1
    
mlab.contour3d(stackedRot)
    

In [ ]:
stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] = stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] + resized_flip


stackedRot_fin = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0

# do for all 10
for kk in range(10):
    M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36*kk, 1.0)

    for ii in range(newSideLen):
        stackedRot_fin[:,:,ii] = ((cv2.warpAffine(stackedRot[:,:,ii], M, (w, h)) > 0.3) * 1 +
                              stackedRot_fin[:,:,ii])

        


In [ ]:
# slice in z-direction
io.imshow(stackedRot_fin[:,:,newSideLen / 2])

In [ ]:
mlab.clf()
plt.close('all')
mlab.contour3d(stackedRot_fin, contours = 10)

In [ ]:
# final density visualization
mlab.clf()
density = stackedRot_fin
grid = mlab.pipeline.scalar_field(density)
min1 = density.min()
max1=density.max()
mlab.pipeline.volume(grid, vmin=min1, vmax=min1 + .5*(max1-min1))

mlab.axes()
mlab.show()

In [ ]:
# visualize with points
# visualize with MayaVi points in 3d
x,y,z = stackedRot_fin.nonzero()

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')
pts = mlab.points3d(x, y, z, stackedRot_fin[x,y,z], colormap = 'hot', 
                    scale_mode='none', scale_factor=1.5, opacity = 0.02)
mlab.axes()
mlab.show()

In [ ]:
io.imshow(resized)
intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10.0
plt.scatter(intPoints[:, 0], intPoints[:,1])

# Resampling and replotting

In [ ]:
# read in all images in direcory, resize, and make a big numpy array
# read in image and resize

newSideLen = 200
img = np.array(cv2.imread(photoList[1], 0))
r = float(newSideLen) / img.shape[1]
dim = (int(newSideLen), int(img.shape[0] * r))

stackedImgs = np.ones((newSideLen, newSideLen, len(photoList)))


for hh in range(len(photoList)):
    img = np.array(cv2.imread(photoList[hh], 0)) # read in image black and white

    # perform the resizing
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    # convert to binary
    resized = np.array((resized < 255) * 1.0)

    # clear edges of resized image (grid from graph)
    resized[:, 0:newSideLen/20] = 0
    resized[newSideLen - newSideLen/20:newSideLen, :] = 0
    
    # add to stacked array
    stackedImgs[:,:,hh] = resized

In [ ]:
def resampRotate(stackedImgs):
    '''
    Resample all the images (same number of photos that I took)
    Randomly rotate images in 1/10 of 360 degree increments
    Put into 3D space
    '''
    
    # prepare to resample resized images
    pollWidth = newSideLen / 100 / 2
    midpt = newSideLen / 2

    # make temporary array
    stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
    w,h = stackedRot.shape[:2]


    # make final array
    stackedRot_fin = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
    
    
    # resample all images
    photoSamp = np.random.choice(range(len(photoList)), size = len(photoList), replace = True)
    # resample rotation angles
    angles = np.random.choice(range(10), len(photoList), True)


    for kk in photoSamp:

        # get image to rotate
        resized = stackedImgs[:,:,kk]
        resized_flip = np.rot90(resized, 3)

        # get rotatation matrix
        M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36*angles[kk], 1.0)

        # reset blank 3D array
        stackedRot = stackedRot * 0.0

        # get image to rotate into 3D space
        stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] = stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] + resized_flip

        # rotate image in 3D, by doing one level at a time
        for ii in range(newSideLen):
            stackedRot_fin[:,:,ii] = ((cv2.warpAffine(stackedRot[:,:,ii], M, (w, h)) > 0.3) * 1 +
                                  stackedRot_fin[:,:,ii])
        
    return stackedRot_fin

In [ ]:
finArry = resampRotate(stackedImgs=stackedImgs)
# mlab.clf()
# mlab.contour3d(finArry, contours = 10)
print np.max(finArry)

In [ ]:
stackedExample  = np.array([stackedImgs[:,:,1] for ii in range(stackedImgs.shape[2]) ])
np.max(stackedExample.sum(axis = 0)) # example for "theoretical maximum"

In [ ]:
# repeat the resampling method multiple times

res_fin = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0

stt = time.time()
for ii in range(500):
    tmp = resampRotate(stackedImgs)
    res_fin = res_fin + tmp
    print ii, time.time() - stt # takes about 8 seconds each

In [ ]:
# save numpy array, so I don't have to keep running the simulation
outfile = '/Users/callinswitzer/Desktop/array5.npy'
np.save(outfile, res_fin)

# start here, if not conducting the simulations

In [368]:
# load np array
res_fin = np.load('/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/array5.npy')
intPoints = np.round(np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10.0)
density = res_fin / (29 * 500) # dividing array by max possible value (29 videos, 500 resamples)

In [445]:
#### mlab.close(all = True)
mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(res_fin, contours= 30)
mlab.axes()

# intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10
# mlab.points3d([100, 100, 100], intPoints[:,0], 200 - intPoints[:,1], colormap = 'hot', 
#                     scale_mode='none', scale_factor=5, opacity = 1)

In [ ]:
mlab.close(all = True)
mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))

# density = res_fin / np.max(res_fin)
grid = mlab.pipeline.scalar_field(density)
min1 = density.min()
max1=density.max()
mlab.pipeline.volume(grid, vmin=min1, vmax=min1 + .5*(max1-min1))
intPoints = (np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10) + 0.5
mlab.points3d([100, 100, 100], intPoints[:,0], 200 - intPoints[:,1], colormap = 'hot', 
                    scale_mode='none', scale_factor=5, opacity = 1)

mlab.axes(extent = [25, 175, 25, 175, 15, 175])
#mlab.show()

In [ ]:
mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))

density = res_fin / np.max(res_fin)
grid = mlab.pipeline.scalar_field(density)
min1 = density.min()
max1=density.max()
mlab.pipeline.volume(grid, vmin=min1, vmax=min1 + .5*(max1-min1))
intPoints = (np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10) + 0.5
mlab.points3d([100, 100, 100], intPoints[:,0], 200 - intPoints[:,1], colormap = 'cool', 
                    scale_mode='none', scale_factor=5, opacity = 1)

mlab.axes(extent = [25, 175, 25, 175, 15, 175])
#mlab.show()

In [ ]:
# visualize with points
# visualize with MayaVi points in 3d
x,y,z = res_fin.nonzero()

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')
pts = mlab.points3d(x, y, z, res_fin[x,y,z], colormap = 'hot', 
                    scale_mode='none', scale_factor=1.5, opacity = 0.02)
mlab.axes()
mlab.show()

In [ ]:
# this way is great!

mlab.close(all = True)
# density = res_fin / np.max(res_fin)

fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(density, contours= 50, opacity = 0.1, colormap = 'hot')
#mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
mlab.orientation_axes()
mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
                    scale_mode='none', scale_factor=2, opacity = 1)

In [ ]:
# this way is great!

mlab.close(all = True)


fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(density, contours= 50, opacity = 0.2, colormap = "jet")
#mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
mlab.orientation_axes()
mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
                    scale_mode='none', scale_factor=2, opacity = 1)

In [ ]:
# contour with matplotlib of a single slice
# make contour figure for paper
os.chdir("/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/Manuscript/DatasetsSupplemental/KalmiaDigitizedPolygon/colormap")
import matplotlib.cm as cm
import colormaps as cmaps


saveDir = '/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/Manuscript/Media/'

# mlab.close(all = True)
density = res_fin / (29 * 500) 

# this cuts the array in half, so we can see inside
Z = np.rot90(density[100, :,:], 3)
#Z /= np.max(Z,axis=(0,1))

# Z[Z>0.4]= 0.41

Z = Z[30:142, 10:190]

levels =  [np.min(Z[np.nonzero(Z)]),0.02, 0.05, 0.11, 0.4, np.max(Z)]

plt.close("all")

fig, ax = plt.subplots()
ax.axis('equal')
ax.axis('off')

colormap_r = matplotlib.colors.ListedColormap(cmaps.magma.colors[::-1])

CF = ax.contourf(Z, levels, cmap=colormap_r)
cbar = fig.colorbar(CF, ticks = levels, fraction = 0.031, pad = 0)
cbar.ax.set_yticklabels([str(np.round(np.min(Z[np.nonzero(Z)]), 3)), "0.02", "0.05", "0.11", "0.4", str(np.round(np.max(Z), 2))])

# plot the contour lines
# using gray scale
CL = ax.contour(Z, levels,
                 linewidths=.5, colors = (0.1, 0.1, 0.1))

#ax.clabel(CL, inline=1, fontsize=14, colors = ["black", 'black', "black", "black", "white"])

plt.savefig(saveDir + "ContourMap.png", dpi = 200)
plt.savefig(saveDir + "ContourMap.pdf")
plt.show()

In [ ]:
# Visualize with slice cut into array -- make into a movie
newSideLen = 200
os.chdir('/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/KalmiaProject/KalmMovie3')

for ctr in range(40):
    
    mlab.close(all = True)
    density = res_fin / (29 * 500)
    # density[:, 102:newSideLen, (5 * ctr):newSideLen] = 0
    density[(5 * ctr):newSideLen, 102:newSideLen, :] = 0

    fig = mlab.figure(size = (1024,768),
                bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
    #mlab.contour3d(density, contours= [0.001, 0.05, 0.2, 0.5], opacity = 0.2, colormap = "jet")
    mlab.contour3d(density, contours= 50, opacity = 0.2, colormap = "hot")
    #mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
    mlab.orientation_axes()
    colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')
    colorbar.scalar_bar_representation.position = [0.8, 0.2]
    colorbar.scalar_bar_representation.position2 = [0.05, 0.6]
#     mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
#                         scale_mode='none', scale_factor=2, opacity = 1)

    mlab.savefig(str(ctr + 41).zfill(3) + '.png', size = (1024, 1024))
    print ctr

In [ ]:
# Visualize with slice cut into array -- make into a movie
newSideLen = 200
os.chdir('/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/KalmiaProject/KalmMovie3')

for ctr in range(40):
    
    mlab.close(all = True)
    density = res_fin / (29 * 500)
    density[:, 102:newSideLen, (5 * ctr):newSideLen] = 0
    #density[(5 * ctr):newSideLen, 102:newSideLen, :] = 0

    fig = mlab.figure(size = (1024,768),
                bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
    #mlab.contour3d(density, contours= [0.001, 0.05, 0.2, 0.5], opacity = 0.2, colormap = "jet")
    mlab.contour3d(density, contours= 50, opacity = 0.2, colormap = "hot")
    #mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
    mlab.orientation_axes()
    colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')
    colorbar.scalar_bar_representation.position = [0.8, 0.2]
    colorbar.scalar_bar_representation.position2 = [0.05, 0.6]
#     mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
#                         scale_mode='none', scale_factor=2, opacity = 1)

    mlab.savefig(str(ctr).zfill(3) + '.png', size = (1024, 1024))
    print ctr

In [ ]:
# Visualize with slice cut into array -- make into a movie
newSideLen = 200
os.chdir('/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/KalmiaProject/KalmMovie3')

# redo 101
for ctr in range(40):
    
    mlab.close(all = True)
    density = res_fin / (29 * 500)
    #density[:, 102:newSideLen, (5 * ctr):newSideLen] = 0
    #density[(5 * ctr):newSideLen, 102:newSideLen, :] = 0
    density[:, :, (5 * ctr):newSideLen] = 0

    fig = mlab.figure(size = (1024,768),
                bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
    #mlab.contour3d(density, contours= [0.001, 0.05, 0.2, 0.5], opacity = 0.2, colormap = "jet")
    mlab.contour3d(density, contours= 50, opacity = 0.2, colormap = "hot")
    #mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
    mlab.orientation_axes()
    colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')
    colorbar.scalar_bar_representation.position = [0.8, 0.2]
    colorbar.scalar_bar_representation.position2 = [0.05, 0.6]
#     mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
#                         scale_mode='none', scale_factor=2, opacity = 1)

    mlab.savefig(str(ctr + 80).zfill(3) + '.png', size = (1024, 1024))
    print ctr

In [ ]:
plt.clf()
plt.hist(np.ndarray.flatten(density), bins = 200)
plt.xlim((0.01, 1))
plt.ylim((0, 10000))

In [ ]:

mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
photoList = [ x for x in onlyfiles if not x.startswith('.') and x.endswith('.png')]

In [ ]:
photoList

In [ ]:
newNames = [str(xx).zfill(3) + '.png' for xx in range(len(photoList))]

In [ ]:
ff = [os.rename(photoList[ii], newNames[ii]) for ii in range(len(photoList))]

In [ ]:
!ffmpeg -framerate 3 -start_number 80 -i %03d.png -c:v libx264 -r 30 -pix_fmt yuv420p 3dVis2_WholeArea.mp4

In [ ]:

!ffmpeg -framerate 3 -start_number 0 -i %03d.png -c:v libx264 -r 30 -pix_fmt yuv420p 3dVis2_Slow.mp4

In [ ]:

!ffmpeg -framerate 10 -start_number 00 -i %03d.png -c:v libx264 -r 30 -pix_fmt yuv420p 3dVis2_Fast.mp4

In [ ]:
# save a figure 

mlab.close(all = True)
density = res_fin / (29 * 500)
#density[:, 102:newSideLen, (5 * ctr):newSideLen] = 0
#density[(5 * ctr):newSideLen, 102:newSideLen, :] = 0
density[101:200, :, :] = 0

fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
#mlab.contour3d(density, contours= [0.001, 0.05, 0.2, 0.5], opacity = 0.2, colormap = "jet")
mlab.contour3d(density, contours= 50, opacity = 0.2, colormap = "hot")
#mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
mlab.orientation_axes()
colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')
colorbar.scalar_bar_representation.position = [0.8, 0.2]
colorbar.scalar_bar_representation.position2 = [0.05, 0.6]
#     mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
#                         scale_mode='none', scale_factor=2, opacity = 1)

# mlab.savefig('halfSizeKalmiaMap' + '.png', size = (1440, 1024))

In [ ]:
# contour surface
mlab.close(all = True)
fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour_surf(density[100, :,:], contours = 50, colormap = 'hot')
colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')

In [401]:
# contour surface
mlab.close(all = True)
fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
aa = mlab.contour_surf(density[100, :,:], contours = [0.009,0.02, 0.05, 0.11, 0.4], colormap = 'hot')
colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')

# mlab.points3d([0, 0, 0], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
#                     scale_mode='none', scale_factor=2, opacity = 1)
#mlab.axes()

mlab.points3d( 100 - intPoints[:,0], 99 - intPoints[:,1] + 1, [0, 0, 0], colormap = 'hot', 
                    scale_mode='none', scale_factor=2, opacity = 1)

In [388]:
vtkout = aa.contour.contour_filter.output.points
vv = vtkout.to_array().T

In [389]:
vv.shape

(3, 2833)

In [397]:
plt.clf()
io.imshow(resized)
plt.scatter(vv[0, :] + 100, -vv[1, :] + 100)
plt.scatter(intPoints[:,0], intPoints[:, 1])

In [370]:
# save a figure 

mlab.close(all = True)
density = res_fin / (29 * 500)
#density[:, 102:newSideLen, (5 * ctr):newSideLen] = 0
#density[(5 * ctr):newSideLen, 102:newSideLen, :] = 0
density[101:200, :, :] = 0

fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
#mlab.contour3d(density, contours= [0.001, 0.05, 0.2, 0.5], opacity = 0.2, colormap = "jet")
mlab.contour3d(density, contours= 50, opacity = 0.2, colormap = "hot")
#mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
mlab.orientation_axes()
colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')
colorbar.scalar_bar_representation.position = [0.8, 0.2]
colorbar.scalar_bar_representation.position2 = [0.05, 0.6]
#     mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
#                         scale_mode='none', scale_factor=2, opacity = 1)

# mlab.savefig('halfSizeKalmiaMap' + '.png', size = (1440, 1024))

In [380]:
#mlab.close(all = True)

yy = vv[0, :] + 101
zz = vv[1, :] + 101
xx = np.ones(yy.shape[0])  + 100
#mlab.axes()

mlab.points3d(xx, yy, zz, scale_factor = 1,line_width = 4.5)
mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], color = (0,0,0), 
                    scale_mode='none', scale_factor=2, opacity = 1)


In [373]:
plt.clf()
plt.scatter(yy, zz)
plt.scatter(intPoints[:,0] + 1, 200 - intPoints[:,1])

In [ ]:
n_mer, n_long = 6, 11
pi = np.pi
dphi = pi / 1000.0
phi = np.arange(0.0, 2 * pi + 0.5 * dphi, dphi)
mu = phi * n_mer

x = np.cos(mu) * (1 + np.cos(n_long * mu / n_mer) * 0.5)
y = np.sin(mu) * (1 + np.cos(n_long * mu / n_mer) * 0.5)
z = np.sin(n_long * mu / n_mer) * 0.5

mlab.plot3d(x, y, z)

In [ ]:
mlab.close(all = True)
mlab.surf(density[100, :,:], warp_scale = 'auto')

In [ ]:
mlab.close(all = True)
mlab.plot3d(xx, yy, zz, tube_radius = 5)

In [424]:
%qtconsole

In [425]:
intPoints

array([[  77.,  169.],
       [ 100.,  169.],
       [ 123.,  169.]])

In [ ]:
100 